In [1]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

c:\Users\bpadmin\anaconda3\envs\tensorflow_cuda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import tensorflow_recommenders as tfrs

In [3]:
ratings = tfds.load("movielens/100k-ratings", split="train")

ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "timestamp": x["timestamp"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"]
})

In [4]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [5]:
type(train)

tensorflow.python.data.ops.dataset_ops.TakeDataset

In [6]:
train.save("../../data/ratings_train")
test.save("../../data/ratings_test")
shuffled.save("../../data/ratings_all")

In [7]:
train_loaded = tf.data.Dataset.load("../../data/ratings_train")
type(train_loaded)

tensorflow.python.data.ops.load_op._LoadDataset

In [8]:
movie_titles = ratings.batch(1_000_000).map(lambda x: x["movie_title"])
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

# need these to initialize timestamp embedding layers in future steps

timestamps = np.concatenate(list(ratings.map(lambda x: x["timestamp"]).batch(100)))

max_timestamp = timestamps.max()
min_timestamp = timestamps.min()

timestamp_buckets = np.linspace(
    min_timestamp, max_timestamp, num=1000,
)

In [9]:
# timestamps = np.concatenate(list(ratings.map(lambda x: x["timestamp"]).batch(100)))
# np.concatenate(list(ratings.map(lambda x: x["timestamp"])))
list((ratings.map(lambda x: x["timestamp"])).batch(100))

[<tf.Tensor: shape=(100,), dtype=int64, numpy=
 array([879024327, 875654590, 882075110, 883326919, 891409199, 876346551,
        882064434, 875135363, 880844102, 884801053, 891636399, 886288624,
        879147917, 877724603, 877887002, 889069782, 880913800, 887667681,
        888466614, 876538595, 880167435, 879439104, 875073408, 884922272,
        890987617, 891439439, 876869445, 891639552, 877319414, 875312441,
        884153606, 888206689, 892679439, 882813761, 879448599, 891239813,
        880696196, 888737277, 892685224, 875296104, 875728913, 889334373,
        883789425, 880139381, 893213019, 887564613, 888904985, 882347699,
        883790609, 881957250, 885943818, 879535752, 891216526, 879875278,
        881107817, 882961804, 891628392, 891217851, 889493069, 892550170,
        879618849, 879975586, 892133310, 891548920, 876348901, 880564534,
        891445354, 891286256, 878184087, 879792557, 876546920, 879310966,
        876526518, 891388286, 880672855, 891386369, 875935609, 87

In [10]:
'''
this handles embedding user Identifiers and contextual data.
time stamp is used as the contexual information here.
using timestamp is 
'''

class UserModel(tf.keras.Model):
    def __init__(self, use_timestamp):
        super().__init__()

        self.use_timestamp = use_timestamp

        self.embed_user_id = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary = unique_user_ids,
                mask_token =None
            ),
            tf.keras.layers.Embedding(
                input_dim = len(unique_user_ids)+1,
                output_dim = 32
            )
        ])

        if self.use_timestamp:
            self.embed_timestamp = tf.keras.Sequential([
                tf.keras.layers.Discretization(
                    bin_boundaries = list(timestamp_buckets)
                ),

                tf.keras.layers.Embedding(
                    input_dim = len(list(timestamp_buckets))+1 ,
                    output_dim = 32
                )
            ])

            self.normalize_timestamp = tf.keras.layers.Normalization(
                axis = None #calcuate a scaler mean and variance 
            )
            self.normalize_timestamp.adapt(timestamps)

    
    def call(self, inputs):

        user_id, timestamp = inputs

        if self.use_timestamp:
            user_id_embed = self.embed_user_id(user_id)
            timestamp_embed = self.embed_timestamp(timestamp)
            norm_timestamp = tf.reshape(self.normalize_timestamp(timestamp), (-1,1)) #(-1,1) means first dimension to be infered

            return tf.concat([user_id_embed, timestamp_embed, norm_timestamp], axis = 1) #concatenate vertically
            
        return self.embed_user_id(user_id)

In [11]:
'''
this handles embedding item Identifiers and contextual data.
movie title itself is used as the contexual information here.
using timestamp is 
'''

class ItemModel(tf.keras.Model):
    def __init__(self):
        super().__init__()

        self.max_tokens = 10000

        self.embed_item_id = tf.keras.Sequential([
            tf.keras.layers.StringLookup(
                vocabulary = unique_movie_titles,
                mask_token =None
            ),
            tf.keras.layers.Embedding(
                input_dim = len(unique_movie_titles)+1,
                output_dim = 32
            )
        ])


        self.textvectorizer = tf.keras.layers.TextVectorization(
            max_tokens = self.max_tokens
        )

        self.embed_item_title = tf.keras.Sequential([
            self.textvectorizer,

            tf.keras.layers.Embedding(
                input_dim = self.max_tokens,
                output_dim = 32,
                mask_zero = True
            ),

            tf.keras.layers.GlobalAveragePooling1D() # reduces dimensionality to 1d (embedding layer embeddeds each word in a title one by one)
        ])

        self.textvectorizer.adapt(unique_movie_titles)
    
    def call(self, inputs):

        movie_title = inputs

        return tf.concat([
            self.embed_item_id(movie_title),
            self.embed_item_title(movie_title)
        ],
        axis = 1)
        
        # return self.embed_item_title(inputs['movie_title'])

In [12]:
class RankingModel(tf.keras.Model):

  def __init__(self, use_timestamp):
    super().__init__()

    # embedding_dimension = 32
    self.use_timestamp = use_timestamp

    self.user_embeddings = UserModel(use_timestamp = self.use_timestamp)

    self.movie_embeddings = ItemModel()

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
  ])

  def call(self, inputs):

    user_id, timestamp, movie_title = inputs

    user_embedding = self.user_embeddings((user_id,timestamp))
    movie_embedding = self.movie_embeddings(movie_title)

    return self.ratings(tf.concat([user_embedding, movie_embedding], axis=1))

In [13]:
RankingModel(use_timestamp= True)((["42"],[886479771], ["One Flew Over the Cuckoo's Nest (1975)"]))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.00831264]], dtype=float32)>

In [14]:
class MovielensModel(tfrs.models.Model):

  def __init__(self, use_timestamp):
    super().__init__()
    self.use_timestamp = use_timestamp
    self.ranking_model: tf.keras.Model = RankingModel(use_timestamp = self.use_timestamp)
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    return self.ranking_model(
        (features["user_id"], features['timestamp'], features["movie_title"]))

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    labels = features.pop("user_rating")

    rating_predictions = self(features)

    # The task computes the loss and the metrics.
    return self.task(labels=labels, predictions=rating_predictions)

In [15]:
model = MovielensModel(use_timestamp = True)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [16]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [19]:
next(cached_train.take(1).as_numpy_iterator())

{'movie_title': array([b'Godfather, The (1972)', b'Escape to Witch Mountain (1975)',
        b'Fargo (1996)', ..., b'Picture Perfect (1997)', b'Fear (1996)',
        b'Sleepers (1996)'], dtype=object),
 'timestamp': array([880859493, 882386848, 879442537, ..., 879481643, 892784130,
        888479151], dtype=int64),
 'user_id': array([b'424', b'429', b'53', ..., b'351', b'551', b'872'], dtype=object),
 'user_rating': array([4., 2., 5., ..., 3., 5., 4.], dtype=float32)}

In [21]:
type(cached_train)

tensorflow.python.data.ops.dataset_ops.CacheDataset

In [20]:
model.fit(cached_train, epochs=10)

Epoch 1/10
10/10 [==============================] - 1s 40ms/step - root_mean_squared_error: 2.3443 - loss: 5.0289 - regularization_loss: 0.0000e+00 - total_loss: 5.0289
Epoch 2/10
10/10 [==============================] - 0s 13ms/step - root_mean_squared_error: 1.1349 - loss: 1.2837 - regularization_loss: 0.0000e+00 - total_loss: 1.2837
Epoch 3/10
10/10 [==============================] - 0s 13ms/step - root_mean_squared_error: 1.1143 - loss: 1.2397 - regularization_loss: 0.0000e+00 - total_loss: 1.2397
Epoch 4/10
10/10 [==============================] - 0s 13ms/step - root_mean_squared_error: 1.1021 - loss: 1.2126 - regularization_loss: 0.0000e+00 - total_loss: 1.2126
Epoch 5/10
10/10 [==============================] - 0s 13ms/step - root_mean_squared_error: 1.0878 - loss: 1.1809 - regularization_loss: 0.0000e+00 - total_loss: 1.1809
Epoch 6/10
10/10 [==============================] - 0s 13ms/step - root_mean_squared_error: 1.0705 - loss: 1.1434 - regularization_loss: 0.0000e+00 - total

In [55]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 1s 25ms/step - root_mean_squared_error: 0.9756 - loss: 0.9494 - regularization_loss: 0.0000e+00 - total_loss: 0.9494


{'root_mean_squared_error': 0.9756286144256592,
 'loss': 0.9386831521987915,
 'regularization_loss': 0,
 'total_loss': 0.9386831521987915}

In [58]:
test_ratings = {}
test_movie_titles = ["M*A*S*H (1970)", "Dances with Wolves (1990)", "Speed (1994)"]
for movie_title in test_movie_titles:
  test_ratings[movie_title] = model({
      "user_id": np.array(["42"]),
      "timestamp": np.array([886479771]),
      "movie_title": np.array([movie_title])
  })

print("Ratings:")
for title, score in sorted(test_ratings.items(), key=lambda x: x[1], reverse=True):
  print(f"{title}: {score}")

Ratings:
M*A*S*H (1970): [[3.9183755]]
Dances with Wolves (1990): [[3.6211972]]
Speed (1994): [[3.5145564]]
